In [1]:
import pandas as pd
import twint
import nest_asyncio
import re


In [2]:
# Solve compatibility issues with notebooks and RunTime errors.
nest_asyncio.apply()

In [3]:
def get_playlist_id(row):
    regex = re.findall(r'\bplaylist\b\/\w*', row)
    if regex:
        return regex[0][9:]
def scrape_playlists(until=None):
    c = twint.Config()
    c.Pandas = True
    c.Pandas_clean = True
    c.Hide_output = True
    c.Limit = 100000
    c.Until = until
    c.Search = 'open.spotify.com/playlist/'
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    until = df.iloc[-1].date[:10]
    df['playlist'] = df.tweet.apply(get_playlist_id)
    df.playlist.to_csv(f'all_playlist_ids{until}.csv', mode='a+', header=False)
    return until
    
# scrape_playlists()
def get_tweets(tid):
    c = twint.Config()
    c.Pandas = True
    c.Pandas_clean = True
    c.Hide_output = True
    c.Lang = 'en'
    c.Limit=100000
    c.Search = f'open.spotify.com/track/{tid}'
    df = twint.storage.panda.Tweets_df
    tweets=  df.tweet
    geos = df.geo
    return zip(list(df.tweet), list(geos))

In [4]:
until = '2018-09-09'

In [5]:
#started 01-09
i = 0
while i < 10:
    until = scrape_playlists(until)
    print(f'scraped until {until}')

scraped until 2018-09-07
scraped until 2018-09-02


IndexError: single positional indexer is out-of-bounds

In [2]:
playlists = pd.read_csv('new_scraped_playlists.csv')

In [3]:
songs = pd.read_csv('scraped_song_ids.csv')

In [4]:
playlists.shape

(16247169, 6)

In [5]:
joined = pd.merge(playlists, songs, left_on='song_id', right_on='song_id', how='left')

In [6]:
missing = joined[joined.artist.isna()].copy()

In [7]:
missing.shape

(2040661, 10)

In [8]:
missing.song_id.to_csv('missing_songs.csv')

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
